# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [1]:
import autowig

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [2]:
!conda build -q ../../bin/conda/libbasic -c statiskit
!conda install -y -q libbasic --use-local -c statiskit

BUILD START: libbasic-1.0.0-0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    certifi:      2016.2.28-py36_0          
    libdev:       1.0.0-py36_0     statiskit
    openssl:      1.0.2l-0                  
    path.py:      10.3.1-py36_0             
    pip:          9.0.1-py36_1              
    python:       3.6.2-0                   
    python-scons: 3.0.0-py36_0     statiskit
    pyyaml:       3.12-py36_0               
    readline:     6.2-2                     
    setuptools:   36.4.0-py36_0             
    six:          1.10.0-py36_0             
    sqlite:       3.13.0-0                  
    tk:           8.5.18-0                  
    wheel:        0.29.0-py36_0             
    xz:           5.2.3-0                   
    yaml:         0.1.6-0                   
    zlib:         1.2.11-0                  


latest version is 3.0.15. Run

conda update -n root conda-build

to get the latest ver

Once the headers have been installed in the system, we parse them with relevant compilation flags.

In [3]:
%%time
import sys
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

CPU times: user 132 ms, sys: 20 ms, total: 152 ms
Wall time: 212 ms


Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is suitable.

In [4]:
%%time
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

CPU times: user 64 ms, sys: 0 ns, total: 64 ms
Wall time: 61.6 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [5]:
%%time
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg,
                             module = '../../src/py/_basic.cpp',
                             decorator = '../../src/py/basic/_basic.py',
                             prefix = 'wrapper_')

CPU times: user 48 ms, sys: 4 ms, total: 52 ms
Wall time: 49 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [6]:
%%time
wrappers.write()

CPU times: user 340 ms, sys: 504 ms, total: 844 ms
Wall time: 2.37 s


Here is the list of the generated wrappers (new files).

In [7]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   basic.ipynb
	modified:   ../../src/py/_basic.h
	modified:   ../../src/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	basic/

no changes added to commit (use "git add" and/or "git commit -a")


And here, we present the wrappers for the `BinomialDistribution` class.

In [8]:
!pygmentize basic/src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [9]:
!conda build -q basic/conda/python-basic -c statiskit
!conda install -y -q python-basic --use-local -c statiskit --force



Package plan for installation in environment /home/main/miniconda:

The following NEW packages will be INSTALLED:

    python-basic: 1.0.0-py27_0 statiskit



Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [10]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [11]:
binomial.pmf(0)

0.5

In [12]:
binomial.pmf(1)

0.5

In [13]:
binomial.n = 0
binomial

In [14]:
binomial.pmf(0)

1.0

In [15]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print error.message
else:
    raise Exception('A `basic.ProbabilityError` should have been raise')

a probability must be in the interval [0,1]


/home/main/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6
  after removing the cwd from sys.path.
